In [1]:
import pandas, numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
import json
import scipy.optimize

In [2]:
q0 = 79.53 # [A^2]
r0 = 66.69 # [A^3]
z_coordination = 10
c_hb = 85580.0 # kcal A^4 / mol/e^2
R = 8.3144598/4184 # 0.001987 # but really: 8.3144598/4184
sigma_hb = 0.0084
EPS = 3.667 # (LIN AND SANDLER USE A CONSTANT FPOL WHICH YIELDS EPS=3.68)
AEFFPRIME = 7.5
EO = 2.395e-4
FPOL = (EPS-1.0)/(EPS+0.5)
ALPHA = (0.3*AEFFPRIME**(1.5))/(EO)
alpha_prime = FPOL*ALPHA
print(alpha_prime, R)

16466.721761795383 0.001987203585086042


In [3]:
sigma_tabulated = np.linspace(-0.025, 0.025, 51)
sigma_m = np.tile(sigma_tabulated,(len(sigma_tabulated),1))
sigma_n = np.tile(np.array(sigma_tabulated,ndmin=2).T,(1,len(sigma_tabulated)))
sigma_acc = np.tril(sigma_n) + np.triu(sigma_m,1)
sigma_don = np.tril(sigma_m) + np.triu(sigma_n,1)
DELTAW = (alpha_prime/2)*(sigma_m + sigma_n)**2+ c_hb*np.maximum(0, sigma_acc - sigma_hb)*np.minimum(0, sigma_don+sigma_hb)

In [4]:
class _fac(object):
    def __init__(self, j):
        assert(j['using_tau_r'])
        self.T_r = j['T_r']
        self.T_max = j['Tmax']
        self.T_min = j['Tmin']
        self.n = np.array(j['n'])
        self.t = np.array(j['t'])
        self.reducing_value = j['reducing_value']
    
    def psat(self, T):
        theta = 1-T/self.T_r
        RHS = np.dot(self.n,theta**self.t)
        return self.reducing_value*np.exp(self.T_r/T*np.sum(RHS))
    
    def dpsat_dT(self, T):
        im = 0+1j; h = 1e-10
        return (self.psat(T+im*h)/h).imag
        
def psat_factory(fluid):
    # Get the JSON structure from CoolProp
    pS = json.loads(CP.get_fluid_param_string(fluid,'JSON'))[0]['ANCILLARIES']['pS']
    return _fac(pS)
    
Ttest = 300
import CoolProp.CoolProp as CP
for fluid in ['Acetone','Water','Ethanol','Benzene']:
    fac = psat_factory(fluid)
    assert(abs(fac.psat(Ttest)-CP.PropsSI('P','T',Ttest,'Q',0,fluid))/fac.psat(Ttest) < 1e-3)
    AS = CP.AbstractState('HEOS',fluid)
    AS.update(CP.QT_INPUTS, 0, Ttest)
    assert(abs(AS.first_saturation_deriv(CP.iP, CP.iT) - fac.dpsat_dT(Ttest))/fac.dpsat_dT(Ttest) < 1e-3)

In [5]:
def get_sigma_profile(name):
    df = pandas.read_csv('profiles/VT2005/Sigma_Profile_Database_Index_v2.txt', sep = '\t')
    mask = df['Compound Name'] == name
    assert(sum(mask)==1)
    index = int(df[mask]['Index No.'].iloc[0])
    V_COSMO = float(df[mask]['Vcosmo, A3'].iloc[0])
    with open('profiles/VT2005/Sigma_Profiles_v2/VT2005-'+'{0:04d}'.format(index)+'-PROF.txt') as fp:
        dd = pandas.read_csv(fp,names=['sigma [e/A^2]','p(sigma)*A [A^2]'],sep='\s+')
        dd['A'] = dd['p(sigma)*A [A^2]'].sum()
        dd['p(sigma)'] = dd['p(sigma)*A [A^2]']/dd['A']
        return dd, V_COSMO

In [6]:
def sigma_plots():
    profs = [get_sigma_profile(n)[0] for n in ['ACETONE','CYCLOPENTANE']]
    As = [prof['p(sigma)*A [A^2]'].sum() for prof in profs]
    for prof in profs:
        plt.plot(prof['sigma [e/A^2]'], prof['p(sigma)*A [A^2]']/prof['p(sigma)*A [A^2]'].max())
    plt.figure()
    x = [0.235000029,1-0.235000029]
    psigma_mix = sum([x[i]*profs[i]['p(sigma)*A [A^2]'] for i in range(2)])/sum([x[i]*As[i] for i in range(2)])
    plt.plot(profs[0]['sigma [e/A^2]'], psigma_mix);
sigma_plots()

In [7]:
def get_Gamma(T, psigma):
    """
    Get the value of Γ (capital gamma) for the given sigma profile
    """
    Gamma = np.ones_like(psigma)
    AA = np.exp(-DELTAW/(R*T))*psigma # constant and can be pre-calculated outside of the loop
    for i in range(50):
        Gammanew = 1/np.sum(AA*Gamma, axis=1)
        difference = np.abs((Gamma-Gammanew)/Gamma)
        Gamma = (Gammanew + Gamma)/2
        if np.max(difference) < 1e-8:
            break
        else:
            pass
    return Gamma

def get_lngamma_resid(T, i, psigma_mix, prof, lnGamma_mix = None):
    """
    The residual contribution to ln(γ_i)
    """
    # For the mixture
    if lnGamma_mix is None:
        lnGamma_mix = np.log(get_Gamma(T, np.array(psigma_mix)))
    # For this component
    psigma = np.array(prof['p(sigma)'])
    A_i = prof['A'].iloc[0]
    lnGammai = np.log(get_Gamma(T, psigma))
    lngammai = A_i/AEFFPRIME*np.sum(psigma*(lnGamma_mix - lnGammai))
    return lngammai

def get_lngamma_comb(T, x, i, profs, V_COSMO_A3):
    """ 
    The combinatorial part of ln(γ_i)
    """
    A = np.array([prof['p(sigma)*A [A^2]'].sum() for prof in profs])
    q = A/q0
    r = V_COSMO_A3/r0
    theta_i = x[i]*q[i]/np.dot(x,q)
    phi_i = x[i]*r[i]/np.dot(x,r)
    l = z_coordination/2*(r-q) - (r-1)
    return (np.log(phi_i/x[i])+z_coordination/2*q[i]*np.log(theta_i/phi_i)
           +l[i]-phi_i/x[i]*np.dot(x,l))

def get_lngamma(T, x, i, psigma_mix, profs, V_COSMO_A3, lnGamma_mix = None):
    """ 
    Sum of the contributions to ln(γ_i)
    """
    return (get_lngamma_resid(T, i, psigma_mix, profs[i], lnGamma_mix=lnGamma_mix) 
           + get_lngamma_comb(T, x, i, profs, V_COSMO_A3))

In [8]:
profs = [get_sigma_profile(n)[0] for n in ['ACETONE','CYCLOPENTANE']]
As = [prof['p(sigma)*A [A^2]'].sum() for prof in profs]
x = [0.235,1-0.235]
T = 623.15
psigma_mix = sum([x[i]*profs[i]['p(sigma)*A [A^2]'] for i in range(2)])/sum([x[i]*As[i] for i in range(2)])
profs,V_COSMO_A3 = zip(*[get_sigma_profile(name) for name in ['ACETONE','CYCLOPENTANE']])
V_COSMO_A3 = np.array(V_COSMO_A3)
i = 0
for i in range(2):
    psigma_mix = sum([x[i]*profs[i]['p(sigma)*A [A^2]'] for i in range(2)])/sum([x[i]*As[i] for i in range(2)])
    lnGamma_mix = np.log(get_Gamma(T, np.array(psigma_mix)))
    print(i, get_lngamma_resid(T, i, psigma_mix, profs[i], lnGamma_mix=lnGamma_mix) )
    get_lngamma(T, x, i, np.array(psigma_mix), profs, V_COSMO_A3, lnGamma_mix = lnGamma_mix)

0 0.27806266299604
1 0.02659344956283812
